In [1]:
import os
import requests
import re
from bs4 import BeautifulSoup
import lxml
import os.path
# from selenium import webdriver
# from selenium.common.exceptions import NoSuchElementException,WebDriverException
# from selenium.webdriver.common.keys import Keys
import time
import warnings
warnings.filterwarnings("ignore")
import pickle
import pandas as pd
from requests.exceptions import ProxyError,ConnectionError,Timeout,HTTPError
import itertools
import multiprocessing
import Ipynb_importer
from IPpool import GetHeaders
from IPpool import Ip_pool
import random

importing Jupyter notebook from IPpool.ipynb


In [12]:
#***************自定义异常类**************************
class MyException(Exception):
    def __init__(self,message):
        Exception.__init__(self)
        self.message=message

In [92]:
class Crawl_common(object):
    '''
    这是用来爬取大众点评的普通类
    
    函数
    -------
    requests_visit_url(self,url,prox_ip）

    '''
    def __init__(self):
        pass
    
    #-----------requests请求网页，返回一个BeautifulSoup对象---------------------
    def requests_visit_url(self,url,prox_ip=None):
        #headers = {"User-Agent":"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/61.0.3163.100 Chrome/61.0.3163.100 Safari/537.36"}
        headers = {"Accept":"application/json, text/javascript, */*; q=0.01",
                    "Accept-Encoding":"gzip, deflate",
                    "Accept-Language":"zh-CN,zh;q=0.8,en;q=0.6",
                    "Connection":"keep-alive",
                    "Cookie":"_hc.v=f72dbe69-bcfc-a9d4-1f7f-d8daeca1ad7f.1505634421; __utma=1.1801105885.1505634423.1505634423.1505636820.2; __utmz=1.1505634423.1.1.utmcsr=google.com.hk|utmccn=(referral)|utmcmd=referral|utmcct=/; _lxsdk_cuid=15e956d9d5ec8-0a28c942cf39be-2a044871-100200-15e956d9d5fc8; _lxsdk=15e956d9d5ec8-0a28c942cf39be-2a044871-100200-15e956d9d5fc8; _lx_utm=utm_source%3DBaidu%26utm_medium%3Dorganic; __mta=222713764.1505836341840.1507819135553.1507819166825.4; JSESSIONID=D2D3D245867334198A59E676F146C5A8; aburl=1; cy=4; cye=guangzhou; s_ViewType=10; _lxsdk_s=15f2012787a-db4-26a-c8%7C%7C47",
                    "Host":"www.dianping.com",
                    "Referer":"http://www.dianping.com/shop/92374637",
                    #"User-Agent":"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/61.0.3163.100 Chrome/61.0.3163.100 Safari/537.36",
                    "X-Requested-With":"XMLHttpRequest"}
        #获取任意一个请求头
        headers["User-Agent"] = random.choice(GetHeaders().user_agent_list)
      
        is_continue = True
        while is_continue:
            try:
                if prox_ip is None:
                    res = requests.get(url,headers = headers)  #访问url，不设代理ip访问
                else:
                    http = prox_ip[0]
                    ip = prox_ip[1]
                    proxies={http:ip}
                    print(proxies)
                    res = requests.get(url,headers = headers,proxies = proxies) #设代理ip访问
                html =  res.content.decode('utf-8')
                soup  =  BeautifulSoup(html,'lxml')
                is_continue = False 

            except HTTPError as e:
                print("断网")
                time.sleep(2)

            except Timeout as e:
                print("请求超时")
                time.sleep(2)

            except ConnectionError as e:
                print("访问被拒")
                time.sleep(2)
        return soup
    
    #---------缓存结果------------------------
    def save_cache(self,result,filename):
        with open(filename,'wb') as f1:
            pickle.dump(result,f1)
            
    #---------加载缓存-----------------------
    def read_cache(self,filename):
        with open(filename,'rb')as f1:
            result = pickle.load(f1)
        return result
    
    #----------切换路径-------------------
    def get_spc_path(self,father_path,child_path):
        os.chdir(father_path)
        if os.path.exists(child_path):
            os.chdir(child_path)
        else:
            os.mkdir(child_path)
            os.chdir(child_path)
        print("done!!")
        
        
    #---------定位一级分类，爬取二级分类------------------
    def classify(self,city_url,city,big_class):
        soup = requests_visit_url(city_url)
        clf_ls = []
        for i in soup.select("#classfy a"):
            clf_url = i["href"] #分类网址
            clf_tl = i.text.strip() # 分类标题
            clf_ls.append([big_class,city,clf_tl,clf_url])
        return clf_ls
    
    #定位二级分类，爬取区县（行政区）
    def get_region(self,class_ls=None):
        url = class_ls[-1]  #url
        soup = requests_visit_url(url) #访问
        region_ls = [] #地区的列表
        for i in soup.select("#region-nav a"):
            if i.text.strip() != "不限":
                region_url = i["href"] 
                region_tl = i.text.strip()
                region = class_ls[:-1]
                region.extend([region_tl,region_url])
                region_ls.append(region)
        return region_ls
    
    #-----------定位了区县，爬取商圈-----------
    def get_circle(self,region_ls=None):
        url = region_ls[-1]   #url网址
        soup = self.requests_visit_url(url) #访问BeautifulSoup
        circle_ls =[]             
        for i in soup.select("#region-nav-sub a"):  #商圈信息
            if i.span.text.strip()!="不限":   #剔除掉不限的url
                circle_url = i["href"]       #商圈url
                circle_tit = i.span.text.strip()  #商圈名
                circle = region_ls[:-1] 
                circle.extend([circle_tit,circle_url]) 
                circle_ls.append(circle)

        #如果没有商圈的话，就拿原来的url当作是商圈的url
        if circle_ls==[]:
            print(region_ls[-2]+"没有子分类！！")
            region_ls.insert(4,"没有商圈")
            circle_ls.append(region_ls)

        return circle_ls
    
    #---------------获取商家具体信息----------------------
    #获取详情层的数据，如星级用户、浏览人数、均价
    def get_info(store_ls=None,prox_ip=None):

        #首先先保证导入的数据不为空
        if store_ls !=[]:
            url = store_ls[-1] #详情信息表
            soup = requests_visit_url(url,prox_ip)
            print(soup)
            try:
                start = soup.select(".mid-rank-stars")[0]["title"]       #星级用户
            except Exception as e:
                print("start",e)
                start = ''

            try:
                review_count = soup.select("#reviewCount")[0].text  #浏览人数
            except Exception as e:
                review_count = ''
                print("review_coun",e)

            try:
                avgprice = soup.select("#avgPriceTitle")[0].text   #均价
            except Exception as e:
                avgprice = ''
                print("avgprice",e)

            #评分系统
            try:
                score= []
                for j in soup.select("#comment_score .item"):
                    score.append(j.text)
                score = "|".join(score)

            except Exception as e:
                score = ""
                print("score",e)

            try:   
                address = soup.select(".address .item")[0].text.strip() #地址
            except Exception as e:
                address = ''
                print("address出错了",e)

            #电话号码
            try:
                tel =[]
                for i in soup.select(".tel .item"):
                    tel.append(i.text)
                tel = "|".join(tel)

            except:
                tel=""
                print("tel出错了",e)

            #路径
            try:
                path = []
                for i in soup.select(".breadcrumb a"):
                    path.append(i.text.strip())
                path ='>'.join(path)
            except Exception as e:
                print("path出错了",e)
                path =''

            info = store_ls[:]
            info.extend([start,review_count,avgprice,score,address,tel,path])
        return info


SyntaxError: invalid syntax (<ipython-input-92-5a70cf18ca77>, line 103)

In [5]:
ip = Ip_pool(['http://www.xicidaili.com/nn/'])
ip_pool = ip.get_ip_pool(test_url='http://www.dianping.com/shop/26910249',timeout =1)

http://www.xicidaili.com/nn/
{'Accept': 'application/json, text/javascript, */*; q=0.01', 'Accept-Encoding': 'gzip, deflate, br', 'Accept-Language': 'zh-CN,zh;q=0.8', 'Connection': 'keep-alive', 'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8', 'User-Agent': 'Mozilla/5.0 (X11; CrOS i686 2268.111.0) AppleWebKit/536.11 (KHTML, like Gecko) Chrome/20.0.1132.57 Safari/536.11'}
坏ip: ['http', 'http://27.40.131.122:61234']
坏ip: ['http', 'http://218.18.232.29:8080']
坏ip: ['http', 'http://113.237.105.186:9999']
坏ip: ['http', 'http://117.78.37.198:8000']
坏ip: ['http', 'http://111.155.116.208:8123']
坏ip: ['http', 'http://110.73.13.92:8123']
坏ip: ['http', 'http://171.37.152.15:8123']
坏ip: ['http', 'http://110.73.4.45:8123']
坏ip: ['http', 'http://110.73.3.96:8123']
坏ip: ['https', 'https://110.73.43.27:8123']
坏ip: ['http', 'http://59.40.69.11:8010']
坏ip: ['http', 'http://123.234.52.106:9999']
坏ip: ['http', 'http://111.155.116.206:8123']
坏ip: ['https', 'https://61.159.195.6:8118']
坏i

In [6]:
ip_pool

[['https', 'https://27.14.169.253:9999'],
 ['http', 'http://61.135.217.7:80'],
 ['https', 'https://113.108.130.210:808'],
 ['https', 'https://42.237.131.97:80'],
 ['https', 'https://111.231.13.171:80'],
 ['https', 'https://182.90.57.83:8123'],
 ['https', 'https://49.81.250.67:8118'],
 ['https', 'https://110.73.15.234:8123'],
 ['https', 'https://119.51.103.164:8080'],
 ['https', 'https://218.15.25.153:808'],
 ['http', 'http://27.40.138.15:61234'],
 ['https', 'https://182.47.125.86:9999'],
 ['http', 'http://60.21.132.218:63000'],
 ['https', 'https://110.73.30.23:8123'],
 ['https', 'https://110.73.40.249:8123'],
 ['https', 'https://222.93.254.204:8118'],
 ['https', 'https://110.73.12.172:8123'],
 ['https', 'https://180.115.0.148:33097'],
 ['https', 'https://1.182.80.34:80']]

# 加ip测试

In [46]:
%%time
f=result[0:10]
num = 0
while len(f)!=0:
    num+=1
    print(num)
    for i in f:
        p=multiprocessing.Pool(processes=3)
        ls= []
        for i in f:
            prox_ip = random.choice(ip_pool)
            ls.append(p.apply_async(get_info,(i,prox_ip)))
        p.close()
        p.join()

        s = []
        f =[]
        for i in ls:
            a = i.get()
            if a[0][-1]=='':
                f.append(a[:8])
            else:
                s.append(a)


1
{'Accept': 'application/json, text/javascript, */*; q=0.01', 'Accept-Encoding': 'gzip, deflate', 'Accept-Language': 'zh-CN,zh;q=0.8,en;q=0.6', 'Connection': 'keep-alive', 'Cookie': '_hc.v=f72dbe69-bcfc-a9d4-1f7f-d8daeca1ad7f.1505634421; __utma=1.1801105885.1505634423.1505634423.1505636820.2; __utmz=1.1505634423.1.1.utmcsr=google.com.hk|utmccn=(referral)|utmcmd=referral|utmcct=/; _lxsdk_cuid=15e956d9d5ec8-0a28c942cf39be-2a044871-100200-15e956d9d5fc8; _lxsdk=15e956d9d5ec8-0a28c942cf39be-2a044871-100200-15e956d9d5fc8; _lx_utm=utm_source%3DBaidu%26utm_medium%3Dorganic; __mta=222713764.1505836341840.1507819135553.1507819166825.4; JSESSIONID=D2D3D245867334198A59E676F146C5A8; aburl=1; cy=4; cye=guangzhou; s_ViewType=10; _lxsdk_s=15f2012787a-db4-26a-c8%7C%7C47', 'Host': 'www.dianping.com', 'Referer': 'http://www.dianping.com/shop/92374637', 'X-Requested-With': 'XMLHttpRequest', 'User-Agent': 'Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3'}

Process ForkPoolWorker-24:
  File "/home/zhh/anaconda3/lib/python3.6/site-packages/urllib3/connection.py", line 141, in _new_conn
    (self.host, self.port), self.timeout, **extra_kw)
Process ForkPoolWorker-25:
Process ForkPoolWorker-26:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/zhh/anaconda3/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/zhh/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/zhh/anaconda3/lib/python3.6/multiprocessing/queues.py", line 343, in get
    res = self._reader.recv_bytes()
  File "/home/zhh/anaconda3/lib/python3.6/site-packages/urllib3/util/connection.py", line 83, in create_connection
    raise err
  File "/home/zhh/anaconda3/lib/python3.6/site-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
  File "/home/zhh/anaconda

KeyboardInterrupt: 

  File "/home/zhh/anaconda3/lib/python3.6/http/client.py", line 1239, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/home/zhh/anaconda3/lib/python3.6/http/client.py", line 1285, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "/home/zhh/anaconda3/lib/python3.6/http/client.py", line 1234, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "/home/zhh/anaconda3/lib/python3.6/http/client.py", line 1239, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/home/zhh/anaconda3/lib/python3.6/http/client.py", line 1285, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "/home/zhh/anaconda3/lib/python3.6/http/client.py", line 1234, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "/home/zhh/anaconda3/lib/python3.6/http/client.py", line 1026, in _send_output
    self.send(msg)
  File "/

In [3]:
 def get_info(store_ls=None,prox_ip=None):

        #首先先保证导入的数据不为空
        if store_ls !=[]:
            url = store_ls[-1] #详情信息表
            soup = requests_visit_url(url,prox_ip)
        
            if soup.title.text == '403 Forbidden':
                is_ban = True
            else:
                is_ban = False
           
            try:
                start = soup.select(".mid-rank-stars")[0]["title"]       #星级用户
            except Exception as e:
                print("start",e)
                start = ''

            try:
                review_count = soup.select("#reviewCount")[0].text  #浏览人数
            except Exception as e:
                review_count = ''
                print("review_coun",e)

            try:
                avgprice = soup.select("#avgPriceTitle")[0].text   #均价
            except Exception as e:
                avgprice = ''
                print("avgprice",e)

            #评分系统
            try:
                score= []
                for j in soup.select("#comment_score .item"):
                    score.append(j.text)
                score = "|".join(score)

            except Exception as e:
                score = ""
                print("score",e)

            try:   
                address = soup.select(".address .item")[0].text.strip() #地址
            except Exception as e:
                address = ''
                print("address出错了",e)

            #电话号码
            try:
                tel =[]
                for i in soup.select(".tel .item"):
                    tel.append(i.text)
                tel = "|".join(tel)

            except:
                tel=""
                print("tel出错了",e)

            #路径
            try:
                path = []
                for i in soup.select(".breadcrumb a"):
                    path.append(i.text.strip())
                path ='>'.join(path)
            except Exception as e:
                print("path出错了",e)
                path =''

            info = store_ls[:]
            info.extend([start,review_count,avgprice,score,address,tel,path])
        return info,is_ban

In [23]:
 def requests_visit_url(url,prox_ip=None):
        #headers = {"User-Agent":"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/61.0.3163.100 Chrome/61.0.3163.100 Safari/537.36"}
        headers = {"Accept":"application/json, text/javascript, */*; q=0.01",
                    "Accept-Encoding":"gzip, deflate",
                    "Accept-Language":"zh-CN,zh;q=0.8,en;q=0.6",
                    "Connection":"keep-alive",
                    "Cookie":"_hc.v=f72dbe69-bcfc-a9d4-1f7f-d8daeca1ad7f.1505634421; __utma=1.1801105885.1505634423.1505634423.1505636820.2; __utmz=1.1505634423.1.1.utmcsr=google.com.hk|utmccn=(referral)|utmcmd=referral|utmcct=/; _lxsdk_cuid=15e956d9d5ec8-0a28c942cf39be-2a044871-100200-15e956d9d5fc8; _lxsdk=15e956d9d5ec8-0a28c942cf39be-2a044871-100200-15e956d9d5fc8; _lx_utm=utm_source%3DBaidu%26utm_medium%3Dorganic; __mta=222713764.1505836341840.1507819135553.1507819166825.4; JSESSIONID=D2D3D245867334198A59E676F146C5A8; aburl=1; cy=4; cye=guangzhou; s_ViewType=10; _lxsdk_s=15f2012787a-db4-26a-c8%7C%7C47",
                    "Host":"www.dianping.com",
                    "Referer":"http://www.dianping.com/shop/92374637",
                    #"User-Agent":"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/61.0.3163.100 Chrome/61.0.3163.100 Safari/537.36",
                    "X-Requested-With":"XMLHttpRequest"}
        headers["User-Agent"] = random.choice(GetHeaders().user_agent_list)
        #eaders =GetHeaders().getHeaders()
        print(headers)
        is_continue = True
        while is_continue:
            try:
                if prox_ip is None:
                    res = requests.get(url,headers = headers)  #访问url，不设代理ip访问
                else:
                    http = prox_ip[0]
                    ip = prox_ip[1]
                    proxies={http:ip}
                    print(proxies)
                    res = requests.get(url,headers = headers,proxies = proxies) #设代理ip访问
                html =  res.content.decode('utf-8')
                soup  =  BeautifulSoup(html,'lxml')
                is_continue = False 

            except HTTPError as e:
                print("断网")
                time.sleep(2)

            except Timeout as e:
                print("请求超时")
                time.sleep(2)

            except ConnectionError as e:
                print("访问被拒")
                is_continue = True
                time.sleep(2)
        return soup

In [2]:
#加载缓存
def read_cache(filename):
    with open(filename,'rb')as f1:
        result = pickle.load(f1)
    return result

In [7]:
sum_ = 0
for i in range(39):
    result = read_cache("/media/zhh/东方国信/智润安排/移动互联网需求/爬虫/缓存/广州_store_ls_{0}_chunk.pickle".format(i))
    sum_ =sum_+len(result)
print(sum_)

25077


### 测试自动换ip的访问函数

In [89]:
 def requests_visit_url_test(url,is_use_ip =False,timeout=3):
        global ip_pool
        global bad_ip
        #headers = {"User-Agent":"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/61.0.3163.100 Chrome/61.0.3163.100 Safari/537.36"}
        headers = {"Accept":"application/json, text/javascript, */*; q=0.01",
                    "Accept-Encoding":"gzip, deflate",
                    "Accept-Language":"zh-CN,zh;q=0.8,en;q=0.6",
                    "Connection":"keep-alive",
                    "Cookie":"_hc.v=f72dbe69-bcfc-a9d4-1f7f-d8daeca1ad7f.1505634421; __utma=1.1801105885.1505634423.1505634423.1505636820.2; __utmz=1.1505634423.1.1.utmcsr=google.com.hk|utmccn=(referral)|utmcmd=referral|utmcct=/; _lxsdk_cuid=15e956d9d5ec8-0a28c942cf39be-2a044871-100200-15e956d9d5fc8; _lxsdk=15e956d9d5ec8-0a28c942cf39be-2a044871-100200-15e956d9d5fc8; _lx_utm=utm_source%3DBaidu%26utm_medium%3Dorganic; __mta=222713764.1505836341840.1507819135553.1507819166825.4; JSESSIONID=D2D3D245867334198A59E676F146C5A8; aburl=1; cy=4; cye=guangzhou; s_ViewType=10; _lxsdk_s=15f2012787a-db4-26a-c8%7C%7C47",
                    "Host":"www.dianping.com",
                    "Referer":"http://www.dianping.com/shop/92374637",
                    #"User-Agent":"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/61.0.3163.100 Chrome/61.0.3163.100 Safari/537.36",
                    "X-Requested-With":"XMLHttpRequest"}

        #获取任意一个请求头
        headers["User-Agent"] = random.choice(GetHeaders().user_agent_list)
        bad_ip = []  #用来装坏的ip
        
        #*********如果不设置ip的话**********************************
        if is_use_ip == False:
            res = requests.get(url,headers = headers,timeout=timeout)  #访问url，不设代理ip访问
            html =  res.content.decode('utf-8')
            soup  =  BeautifulSoup(html,'lxml')
            if soup.title.text == '403 Forbidden':
                print("ip被禁止了")
                raise ConnectionError
           
        #************如果设置ip的话，则执行这一段*********************
        else:
            is_continue = True   #设置循环标志
            while is_continue:
            #整理成ip地址的格式
                try:
                    #
                    if len(ip_pool)==len(bad_ip):
                        ip_url = ['http://www.xicidaili.com/nn/']
                        test_url = "http://www.dianping.com/shop/2954893"
                        ip_pool= Ip_pool(ip_url).get_ip_pool(test_url)
                        prox_ip = random.choice(ip_pool)
                        bad_ip = []
                        
                    #******随机生成一个ip*******
                    prox_ip = random.choice(ip_pool)
                    http = prox_ip[0]
                    ip = prox_ip[1]
                    proxies={http:ip}
                    print(proxies)
                    
                    #*******用代理ip访问********
                    res = requests.get(url,headers = headers,proxies = proxies,timeout=timeout) #设代理ip访问
                    html =  res.content.decode('utf-8')
                    soup  =  BeautifulSoup(html,'lxml')
            
                    #********如果返回网页被禁止的情况，触发异常,否则返回正常**********
                    if soup.title.text == '403 Forbidden':
                        print("ip被禁止了")
                        raise ConnectionError
                    else:
                        is_continue = False

                except HTTPError as e:
                    bad_ip.append(prox_ip)
                    print("断网")
                    prox_ip = random.choice(ip_pool)


                except Timeout as e:
                    bad_ip.append(prox_ip)
                    print("超时")
                    prox_ip = random.choice(ip_pool)


                except ConnectionError as e:
                    print("访问被拒")
                    prox_ip = random.choice(ip_pool)

        return soup

In [90]:
requests_visit_url_test("http://www.dianping.com/shop/94034183",is_use_ip=True)

{'https': 'https://110.73.5.237:8123'}


<!DOCTYPE html>
<html>
<head>
<link href="http://www.dianping.com/shop/94034183" rel="canonical"/>
<link href="http://m.dianping.com/shop/94034183" media="only screen and (max-width: 640px)" rel="alternate"/>
<script type="text/javascript">(function(){var g=["iPhone","Android","Windows Phone"],b="https://m.dianping.com",e=window.location.pathname+window.location.search,f=false;for(var d=0,a;a=g[d];d++){if(navigator.userAgent.indexOf(a)!=-1){f=true;break}}function h(){location.href=b+e}function c(){if(document.referrer.indexOf(b)==0){document.cookie="vmod=pc;expires=0;"}return document.cookie.match(/vmod=pc/)}if(f){if(!c()){h()}}})();</script>
<title>搭台·烧卤专门店电话,地址,营业时间(图)-广州美食-大众点评网</title>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="搭台·烧卤专门店null,潮汕菜,广州,南村,地址,电话,评论,网友点评,推荐菜,照片/图片,营业时间,公交信息" name="Keywords"/>
<meta content="广州搭台·烧卤专门店:人均价格64元。进入搭台·烧卤专门店页面,查看更多关于搭台·烧卤专门店的地址、电话、菜单、价格、营业时间介绍,了解搭台·烧卤专门店最新的折扣优惠券、用户点评信息。" name="Description"/>
<

In [68]:
def high_requests_visit_url(url,ip_pool = None,timeout=4):
    if  ip_pool ==None:
        soup = requests_visit_url_test(url)
        
    else:
        is_continue=True
        bad_ip=[]
        prox_ip = random.choice(ip_pool)
        while is_continue: 
            try:
                soup = requests_visit_url_test(url,prox_ip=prox_ip,timeout=timeout)
                is_continue = False

            except ConnectionError:
                print("连接失误")
                bad_ip.append(prox_ip)
                prox_ip = random.choice(ip_pool)

            except Timeout:
                bad_ip.append(prox_ip)
                print("超时")
                prox_ip = random.choice(ip_pool)
                
            finally:
                if (len(ip_pool)-len(bad_ip)) == 0:
                    ip_url = ['http://www.xicidaili.com/nn/']
                    test_url = "http://www.dianping.com/shop/2954893"
                    ip_pool= Ip_pool(ip_url).get_ip_pool(test_url)
                    prox_ip = random.choice(ip_pool)
                    bad_ip = []

    return soup

In [71]:
soup = high_requests_visit_url("http://www.dianping.com/shop/94034183",ip_pool)

{'https': 'https://120.78.15.63:80'}


In [7]:
def read_cache(filename):
    with open(filename,'rb')as f1:
        result = pickle.load(f1)
    return result

sum_=0
for i in range(249):
    result = read_cache("/media/zhh/东方国信/智润安排/移动互联网需求/爬虫/缓存/广州_store_ls_{0}_chunk.pickle".format(i))
    sum_ =len(result)+sum_
print(sum_)

56123


[['美食',
  '广州',
  '咖啡厅',
  '海珠区',
  '客村/赤岗',
  'http://www.dianping.com/search/category/4/10/g132r2081',
  'Golden House',
  'http://www.dianping.com/shop/69938449'],
 ['美食',
  '广州',
  '咖啡厅',
  '海珠区',
  '客村/赤岗',
  'http://www.dianping.com/search/category/4/10/g132r2081',
  'BENSHOP工作室',
  'http://www.dianping.com/shop/18244563'],
 ['美食',
  '广州',
  '咖啡厅',
  '海珠区',
  '客村/赤岗',
  'http://www.dianping.com/search/category/4/10/g132r2081',
  'COVITA COFFEE(客村店)',
  'http://www.dianping.com/shop/66685752'],
 ['美食',
  '广州',
  '咖啡厅',
  '海珠区',
  '客村/赤岗',
  'http://www.dianping.com/search/category/4/10/g132r2081',
  'cāo shop',
  'http://www.dianping.com/shop/66954025'],
 ['美食',
  '广州',
  '咖啡厅',
  '海珠区',
  '客村/赤岗',
  'http://www.dianping.com/search/category/4/10/g132r2081',
  'XXL CAFE',
  'http://www.dianping.com/shop/92944966'],
 ['美食',
  '广州',
  '咖啡厅',
  '海珠区',
  '客村/赤岗',
  'http://www.dianping.com/search/category/4/10/g132r2081',
  '意淇',
  'http://www.dianping.com/shop/68953396'],
 ['美食',
  '广